In [157]:
!pip install selenium







In [158]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options




In [159]:

options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'



In [160]:
pip install webdriver_manager


In [161]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.firefox import GeckoDriverManager
import time



# LinkedIn credentials
linkedin_email = "chetnagirish2@gmail.com"
linkedin_password = "Boss1234$"

# Set up GeckoDriver
GeckoDriverManager().install()

'C:\\Users\\chetn\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

In [162]:
# Initialize the Firefox webdriver
options = Options()
driver = webdriver.Firefox(options=options)

In [163]:
import re

def check_element (elem):
    if elem is None: 
        return ''
    if len (elem) > 0:
        return elem.text.strip()
    else:
        return ''



def extract_hourly_or_salary_range(salary_string):
    hourly_match = re.search(r'\$([\d,]+(?:\.\d+)?)\s?(?:-|to)?\s?\$?([\d,]+(?:\.\d+)?)?\s?\/\s?hr', salary_string)
    salary_match = re.search(r'\$([\d,]+(?:\.\d+)?)\s?K?\s?\/\s?yr\s?(?:-|to)?\s?\$?([\d,]+(?:\.\d+)?)?\s?K?\s?\/\s?yr', salary_string)

    if hourly_match:
        hourly_rate_lower = float(hourly_match.group(1).replace(',', ''))
        hourly_rate_upper = float(hourly_match.group(2).replace(',', '')) if hourly_match.group(2) else hourly_rate_lower

        # Assuming 40 hours per week and 52 weeks per year
        annual_lower = hourly_rate_lower * 40 * 52
        annual_upper = hourly_rate_upper * 40 * 52

        average = (annual_lower + annual_upper) / 2

        return [annual_lower, annual_upper, average]
    elif salary_match:
        salary_lower = float(salary_match.group(1).replace(',', '')) * 1000
        salary_upper = float(salary_match.group(2).replace(',', '')) * 1000 if salary_match.group(2) else salary_lower

        average = (salary_lower + salary_upper) / 2

        return [salary_lower, salary_upper, average]
    else:
        return [0,0,0]


In [164]:
# Open LinkedIn login page
driver.get('https://www.linkedin.com/jobs')

In [165]:
email_input = driver.find_element("name", "session_key")
password_input = driver.find_element("name", "session_password")
login_button = driver.find_element("xpath", "//button[@type='submit']")

In [166]:
email_input.send_keys(linkedin_email)
password_input.send_keys(linkedin_password)

In [167]:
login_button.click()

In [168]:
search_box = driver.find_element(By.CLASS_NAME, 'jobs-search-box__text-input')
search_box.send_keys('Data Analyst')
search_box.send_keys(Keys.RETURN)


In [169]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Wait for the element to be visible
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "searchFilter_timePostedRange")))

# Once visible, locate and click on the element
date_filter = driver.find_element(By.ID, "searchFilter_timePostedRange")
date_filter.click()

#date_filter = driver.find_element("id", "searchFilter_timePostedRange")
#date_filter.click()


In [170]:
class_name = "t-14 t-black--light t-normal"
span_text = "week"

past_week_x_path= f"//span[@class='{class_name}' and contains(.,'{span_text}')]"
past_week = driver.find_element("xpath", past_week_x_path)
past_week.click()


In [171]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class_name = "artdeco-button__text"
span_text = "results"
show_results_xpath = f"//span[@class='{class_name}' and contains(.,'{span_text}')]"
# Wait for the element to be clickable
show_results = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, show_results_xpath)))
show_results.click()



In [172]:
jobs_block = driver.find_element(By.CSS_SELECTOR, '.scaffold-layout__list-container')
jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')


In [173]:
# browser.execute_script("arguments[0].scrollIntoView();", target_li_elements[24])
driver.execute_script("arguments[0].scrollIntoView();", jobs_list[len(jobs_list)-1])

In [174]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")


In [175]:
target_class = 'jobs-search-results'
target_li_elements = soup.find_all('li', class_=lambda x: x and 'jobs-search-results__list-item' in x)

In [176]:

num = 1
job_title = check_element(target_li_elements[num].find('a', class_=lambda x: x and 'title' in x))

# Assuming extract_hourly_or_salary_range and target_li_elements are defined elsewhere
company = check_element(target_li_elements[num].find('span', class_='job-card-container__primary-description'))
location = check_element(target_li_elements[num].find('div', class_='artdeco-entity-lockup__caption ember-view'))
salary = check_element(target_li_elements[num].find('div', class_= lambda x: x and 'mt1 t-sans t-12 t-black--light' in x))
link = target_li_elements[num].find('a').get('href')

dic = {
    'job_title': job_title,
    'company': company,
    'location': location,
    'salary_string': salary,
    'lower_bound_extracted_salary': extract_hourly_or_salary_range(salary)[0],
    'upper_extracted_salary': extract_hourly_or_salary_range(salary)[1],
    'average_extracted_salary': extract_hourly_or_salary_range(salary)[2],
    'link': 'www.linkedin.com' + link
}

In [177]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
import time

# Define the XPath expression for the next page button
next_page_xpath = "//button[@aria-label='Page " + str(page) + "']"

# Wait for the next page button to be clickable
wait = WebDriverWait(driver, 10)
next_page = wait.until(EC.element_to_be_clickable((By.XPATH, next_page_xpath)))

# Retry the click with a delay if it fails due to interception
retry_attempts = 3
for attempt in range(retry_attempts):
    try:
        next_page.click()
        break  # Click successful, exit the loop
    except ElementClickInterceptedException:
        print("Click intercepted. Retrying...")
        time.sleep(1)  # Wait for 1 second before retrying
else:
    print("Failed to click after {} attempts".format(retry_attempts))

In [178]:
len (jobs_list)

25

In [179]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Assuming jobs_list is a list of web elements
for i in range(len(jobs_list)):
    print("__________")
    n += 1
    try:
        # Wait for the element to be clickable
        job_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "your_xpath_here")))
        
        # Scroll the element into view
        driver.execute_script("arguments[0].scrollIntoView();", job_element)
        
        # Print the text of the element
        print(n, " ", job_element.text)
    except StaleElementReferenceException:
        # Handle stale element reference exception gracefully
        print("Stale element reference encountered. Retrying...")
        continue

__________


TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16
